# S04_SparkSTFLocatel

In [3]:
import pyspark
print(pyspark.__version__)

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("locatel.txt").getOrCreate()

3.5.3


24/10/12 15:36:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace,date_format
from pyspark.sql.types import IntegerType, DoubleType, FloatType

dir_hdfs:"hdfs://namenode:9000/tmp/amd/locatel0311";

In [ ]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)

In [ ]:
df.printSchema()

print(f"Número de renglones: {df.count()}")
print(f"Número de columnas: {len(df.columns())}")
print(f"Columnas: {df.columns()}")

In [ ]:
df.show(truncate=False)

In [ ]:
datos = df.select(['fecha_solicitud','hora_solicitud','tema_solicitud','sexo',
                   'edad','estatus','alcaldia','colonia_datos','latitud','longitud'])

datos.printSchema()
datos.count()

In [ ]:
datos.filter('sexo = "FEMENINO"').count()

In [ ]:
datos = df.select(['fecha_solicitud','hora_solicitud','tema_solicitud','sexo',
                   'edad','estatus','alcaldia','colonia_datos','latitud','longitud']).filter('sexo = "FEMENINO"').orderBy('fecha_solicitud').show()

In [ ]:
datos = datos.withColumn('fecha_solicitud', to_date(col('fecha_solicitud'),"yyyy-MM-dd")).\
    withColumn('edad',col("fecha_solicitud").cast(FloatType())).\
    withColumn('latitud',col("latitud").cast(DoubleType())).\
    withColumn('longitud',col("longitud").cast(DoubleType()))

datos.printSchema()

In [ ]:
datos.show()

## Clase 27/09/2024

In [ ]:
#Contar nulos
datos.select([sum(col(c).isNull().cast(int)).alias(c) for c in datos.columns()]).show()

In [ ]:
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth

datos_tiempo = datos.select(col("edad"),col("estatus"),col("tema_solicitud"),
                            year(col(col)).alias("anio"))
datos_tiempo.printSchema()
datos_tiempo.describe().show(truncate=False)

In [ ]:
columnas = ['anio','mes','dia','hora','minuto','segundo','semana']
datos_tiempo.select(columnas).describe().show()
datos_tiempo.select(columnas).describe().toPandas()

In [ ]:
#Ver datos únicos
datos_tiempo.select("tema_solicitud").distinct().count()

In [ ]:
datos_tiempo.select("tema_solicitud").distinct().show(truncate=False)

In [ ]:
datos_tiempo.select("tema_solicitud").count().orderBy("tema_solicitud").\
show(100, truncate=False)

In [ ]:
datos_tiempo.select("tema_solicitud").count().orderBy("count",ascending=False).\
show(100,truncate=False)

## Clase 30/09/2024

In [ ]:
#Localhost:4040
datos_tiempo.printSchema()

In [ ]:
datos_tiempo.filter(datos_tiempo.anio==2020).count()

In [ ]:
datos_tiempo.filter(datos_tiempo.anio==2020).show(truncate=False)

In [ ]:
datos_tiempo.select(['edad','anio','nombre_dia','hora','estatus']).filter((datos_tiempo.anio==2020) and (datos_tiempo<=6)).\
    filter(datos_tiempo.nombre_dia =="Sunday").filter(datos_tiempo.edad>0).\
    show()

In [ ]:
datos.groupBy("alcaldia").count().show()

In [ ]:
datos.groupBy("alcaldia").count().show(100,truncate=False)

In [ ]:
datos.groupBy("alcaldia").agg({"edad":"avg"}).show(100, truncate=False)

In [ ]:
dft.write.csv("hdfs://namenode:9000/tmp/amd/locateltiempo")

In [ ]:
type(dft)

# S05_SparkSTFLocatel

In [ ]:
dft.write.mode("").cvs("hdfs://namenode:9000/tmp/amd/locateltiempo")


In [ ]:
type(dft)

In [ ]:
dft.select("*", (dft.edad * 12).alias("edad_meses")).show(100, truncate = False)

In [ ]:
dft.select("*", (dft.edad * 12).\
           alias("edad_meses")write.mode("overwrite").\
            cvs("hdfs://namenode:9000/tmp/amd/locateltiempo", header = True))

In [ ]:
dft.select("*", (dft.edad * 12).\
           alias("edad_meses")write.mode("overwrite").\
            cvs("hdfs://namenode:9000/tmp/amd/locateltiempo", header = True))

In [ ]:
from pyspark.sql.functions import count
datos_tiempo.groupBy("anio", "mes").agg(count("*").alias("nr")).show(truncate = False)

In [ ]:
datos_tiempo.select(["edad","estatus","tema_solicitud", "anio","mes"])\
            .partitionBy("anio")\
            .write.mode("append")\
            .cvs("hdfs://namenode:9000/tmp/amd/locateltiempobyanio", header = True)

In [ ]:
dfp = spark.read.option(header = True)\
           .cvs("hdfs://namenode:9000/tmp/amd/locateltiempobyanioymes")

In [ ]:
dfp.printSchema()

In [ ]:
dfp.show(30, truncate = False)

In [ ]:
dfp.filter('mes = 7 AND anio = 2022').show()

## Clase 07/10/2024

In [ ]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace
from pyspark.sql.functions import date_format
from pyspark.sql.types import IntergerType, DoubleType, FloatType

In [ ]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/locatel0311"

In [ ]:
from pyspark.sql.functions import agg, max, min, count

dfg = datos.groupBy("alcaldia").\
    agg(avg("edad").alias("edad_promedio")
                max("edad").alias("edad_maxima")
                min("edad").alias("edad_minima")
                count("*").alias("no_registros"))

In [ ]:
dfg.groupBy("alcaldia_solicitud",ascendig = False).show(100, truncate=False)

In [ ]:
from pyspark.sql.functions import when

datos = datos.\
    withColumn("alcaldia_solicitud", when(datos.alcaldia_solicitud == "IZTACALCO", "iZTACALCO")).\
    otherwise(datos.alcaldia_solicitud)

In [ ]:
from os import truncate
datos.groupBy("alcaldia").\
    agg(avg("edad").alias("edad_promedio")
                max("edad").alias("edad_maxima")
                min("edad").alias("edad_minima")
                count("*").alias("no_registros")).\
    orderBy("alcaldia_solicitud",ascending = False).\
    show(100,truncate = False)

In [ ]:
sepomex_df = spark.read.\
    option("header", "True").option("delimiter", "|").\
    option("skipRows", "1")sch(schema).\
    csv("hdfs://namenode:9000/tmp/amd/sepomex")

In [ ]:
dfcp = sepomex_df.select(['d_codigo','D_mnpio','d_estado'])

In [ ]:
sepomex_df.printSchema()

In [ ]:
dfcp.count()

In [ ]:
dfcp.show(truncate = False)

In [ ]:
from pyspark.sql.functions import col, coalesce

datos_mapeados = datos.join(dfcp.select(["d_codigo", "D_mnpio"]), datos["codigo_postal_solicitud"] == dfcp["d_codigo"], "left")

In [ ]:
datos_mapeados.printSchema()

In [ ]:
datos_mapeados.select(["codigo_postal_solicitud", "alcaldia_solicitud", "d_codigo", "D_mnpio"]).show(100, truncate = False)

In [ ]:
datos_mapeados.withColumn("alcaldia", coalesce(col("D_mnpio"), col("alcaldia")))

In [ ]:
datos_mapeados.select(["edad", "alcadia", "codigo_postal_solicitud","d_codigo"]).show(100, truncate = False)

In [ ]:
#Verificar duplicados
dfcp_dup = dfcp.groupBy("d_codigo").agg(count("*").alias("count")).\
    filter("count" > 1)

print(dfcp_dup.count())
dfcp_dup.orderBy("count", ascending = True).show(100,truncate = False)

In [ ]:
#ver duplicados
dfcp.filter("d_codigo = 85203").show(truncate = False)

In [ ]:
#Eliminar duplicados
dfcp = dfcp.dropDuplicates("d_codigo")

In [ ]:
#Validar que ya no hay duplicados
dfcp.groupBy("d_codigo").agg(count("*").alias("count")).filter("count > 1").show()

In [ ]:
from pyspark.sql.functions import col, lpad, regexp_replace, trim
datos = datos.withColumn(
    "codigo_postal_solicitud", lpad (
        trim(
          regexp_replace (
              col(
                  "codigo_postal_solicitud"),"[0-9]", "")
        ), 5, "0")
)

In [ ]:
datos.select("codigo_postal_solicitud").show()

In [ ]:
datos.select("codigo_postal_solicitud" == "00000").\
    dropDup(["codigo_postal_solicitud"]).\
    filter("codigo_postal_solicitud") and (len(col("alcaldia")==5)).show()

In [ ]:
#Actualizar codigo postal, cambiarlo por alcaldía
datos = datos.withColumn("codigo_postal_solicitud" when (
         (col("codigo_postal_solicitud") = "00000")
            and (lenght(col("alcaldia")) = 5),
         col("alcaldia_solicitud")
    )
)

In [ ]:
datos.select("alcaldia","codigo_psotal_solicitud").filter(
    col("codigo_postal_solicitud") == col ("alcaldia")
).show()

In [ ]:
dfg = datos_mapeados.groupBy("alcaldia").agg(
    avg("edad").alias("edad_promedio"),
    max("edad").alias("edad_maxima"),
    min("edad").alias("edad_minima"),
    count("*").alias("no_registros")
    )

In [ ]:
dfg.count()

In [ ]:
dfg.orderBy("alcaldia_solicitud",ascendig = False).show(100,truncate = False)

In [ ]:
datos_mapeados.write.mode("overwrite").option("heeader","true").csv("hdfs://namenode:9000/tmp/amd/locatellimpio")